# Closures et Générateurs

Alexandre Bovet

UNamur et UCLouvain

alexandre.bovet@unamur.be


### Liste de fonctions

Continuons la construction d’un code pour le pluriel des mots anglais

- Pour le moment: une liste de règles
- Ajoutons un niveau d’abstraction et compliquons un peu le tout:
 - 1 fonction par recherche de correspondance
 - 1 fonction par substitution



In [ ]:
import re

def match_sxz(noun):
    return re.search('[sxz]$', noun)

def apply_sxz(noun):
    return re.sub('$', 'es', noun)

def match_h(noun):
    return re.search('[^aeioudgkprt]h$', noun)

def apply_h(noun):
    return re.sub('$', 'es', noun)

def match_y(noun):                             
    return re.search('[^aeiou]y$', noun)
        
def apply_y(noun):                             
    return re.sub('y$', 'ies', noun)

def match_default(noun):
    return True

def apply_default(noun):
    return noun + 's'

# Tuple de paires de fonctions (match, apply)
rules = ((match_sxz, apply_sxz),               
         (match_h, apply_h),
         (match_y, apply_y),
         (match_default, apply_default)
         )

# plural() réduit à quelques lignes: boucle sur rules 
# la fonction va toujours retourner quelque chose puisque la
# dernière matche_rule est toujours True
def plural(noun):           
    for matches_rule, apply_rule in rules:      
        if matches_rule(noun):
            return apply_rule(noun)

In [ ]:
# fonction equivalente:
def plural_long(noun):
    if match_sxz(noun):
        return apply_sxz(noun)
    if match_h(noun):
        return apply_h(noun)
    if match_y(noun):
        return apply_y(noun)
    if match_default(noun):
        return apply_default(noun)

##### Liste de patterns
- Définition de fonctions pour chaque règle pas nécessaire
 - On ne les appelle pas directement
 - Fonction similaires: uniquement expressions régulières différentes
 
```python
def match_y(noun):                             
    return re.search('[^aeiou]y$', noun)
        
def apply_y(noun):                             
    return re.sub('y$', 'ies', noun)
```

=> Génération de fonctions et liste de patterns!


In [ ]:
import re

def build_match_and_apply_functions(pattern, search, replace):

    def matches_rule(word):                
        return re.search(pattern, word)


    def apply_rule(word):  
        return re.sub(search, replace, word)


    return (matches_rule, apply_rule)

- `marches_rules` et `apply_rule` sont des **fonctions imbriquées** (nested functions): fonction dont la définition est encapsulée dans une autre fonction.

- Paramètres `pattern`, `search` et `replace` sont passés à une fonctions imbriquées comme constantes (pas comme arguments): **Closure**
 - lorsqu'on appelle `build_match_and_apply_functions`, les variables `pattern`, `search` et `replace` sont gardées en vie même après que la fonction ait retournée.


##### Mise en pratique

In [ ]:
patterns = (
            ('[sxz]$',           '$',  'es'),
            ('[^aeioudgkprt]h$', '$',  'es'),
            ('(qu|[^aeiou])y$',  'y$', 'ies'),
            ('$',                '$',  's')                                
          )
    
rules = [build_match_and_apply_functions(pattern, search, replace)
         for (pattern, search, replace) in patterns]

def plural(noun):
    for matches_rule, apply_rule in rules:
        if matches_rule(noun):
            return apply_rule(noun)

##### Un fichier de patterns

Fichier plural-rules.txt:

```
[sxz]$               $    es
[^aeioudgkprt]h$     $    es
[^aeiou]y$          y$    ies
$  $    s 
```

In [ ]:
rules = []

with open('examples/plural_rules.txt', encoding='utf-8') as pattern_file: 
    for line in pattern_file:                                     
        pattern, search, replace = line.split(None, 3)            
        rules.append(build_match_and_apply_functions(pattern, search, replace))

def plural(noun):
    for matches_rule, apply_rule in rules:
        if matches_rule(noun):
            return apply_rule(noun)        
        

=> Codes et données bien séparées !

### Générateurs

`yield` : définit fonction spéciale generator
- génération de valeurs successives
- 1 valeur à la fois
- pause la fonction


In [ ]:
def make_counter(x): 
    print('entering make_counter') 
    while True: 
        yield x 
        print('incrementing x') 
        x = x + 1 

Création d’une instance du générateur (mais n’exécute pas de code!)

In [ ]:
counter = make_counter(2)
counter

`next` prend un générateur et retourne la valeur suivante -> redémarre la fonction à partir de et jusqu’à `yield`

In [ ]:
next(counter)

##### Générateur: Suite de Fibonacci

In [ ]:
def fib(max):

    # condition initiales
    a, b = 0, 1  
        
    while a < max:
        yield a # retourne la valeur courante a chaque iteration
        a, b = b, a + b

Utilisation d’un générateur dans une boucle `for`:

In [ ]:
for n in fib(1000): 
    print(n, end=' ')

Itération automatique sur tout le générateur et retourne une liste:

In [ ]:
list(fib(1000))

#####  Retour au pluriel des mots

In [ ]:
def rules(rules_filename):
    with open(rules_filename, encoding='utf-8') as pattern_file:
        for line in pattern_file:
            pattern, search, replace = line.split(None, 3)
            yield build_match_and_apply_functions(pattern, search, replace)


- `yield` pour la construction des paires de fonctions
 - `rules` = generateur!
 - => Utilisation dans `for`

In [ ]:
def plural(noun, rules_filename='examples/plural_rules.txt'):
    for matches_rule, apply_rule in rules(rules_filename):
        if matches_rule(noun):
            return apply_rule(noun)
    raise ValueError('no matching rule for {0}'.format(noun)) 


Au 1er appelle à `rules` va ouvrir le ficher, lire la 1ère ligne et retourner les fonctions match et apply. 
Au 2ème appelle, `rules` va reprendre la ou il en était. Le fichier et toujours ouvert.

- Gain: temps, mémoire (on ne lit pas forcement tout le fichier)
- Perte: performance (on ouvre le fichier à chaque appelle de plural)

Solution: construire son propre itérateur!